In [ ]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt


import os
import gc
import sys
srcpath = os.path.realpath('./../libraries')
sys.path.append(srcpath)

#import the basic libraries for Tabor AWT
from tevisainst import TEVisaInst
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from pulse_sequencev2 import EPR_pulse

# import the libraries for the SignalCore unit
import sc5318a
import time
from sc5318a import *

In [ ]:
# Initializing all components

## Tabor AWT
inst_addr = '169.254.18.140' #Proteus Lan
inst = TEVisaInst(inst_addr)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)
paranoia_level = 2 # The paranoia-level is either low (0), normal (1) or debug (2)
# Reset the instrument
inst.send_scpi_cmd('*CLS; *RST')
print ('Reset instrument ..')
dac_mode = 16 

model_name = inst.send_scpi_query('SYST:INF:MODel?') # Get the model
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
# scaling
max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
wpt_type = np.uint16   
# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))
sampleRateDAC = 2.5E9
cmd = ':FREQ:RAST {0}'.format(sampleRateDAC) 
inst.send_scpi_cmd(cmd)
print('Tabor AWT initialized')
resp = inst.query(':SYST:ERR?')
print(resp)

## SignalCore
print("INITIALIZING AND GETTING BASIC DEVICE INFORMATION")
#Establish the name and serial number of the instrument.
SC1 = SignalCore_SC5318A("SC1", "1000491B")
print("Device opened!")
#displays the device's info
get_idn()
print("Device current temperature:{0}".format(device_temp()))

get_rf_frequency()
get_if_frequency()
get_lo_frequency()

print ('Device current RF parameters:')
display_rf_parameters()

#get the gain conversion
calc_gain()

#reading from the eeprom
read_eeprom(0, 0x04, 4)

#example of the reg_read and reg_write commands
#reads back the device's serial number
reg_read(0x30, 0)
#sets RF in freq to 1200000000000 Hz
reg_write(0x10, 1200000000000)

print("Changing the IF frequency value...")
set_if_frequency(1.2e9)
get_if_atten_value()

print("Closing device")
close_device()



